#Load AirBnB Data

In [ ]:
import pandas as pd

# Load the data directly from the raw GitHub URL
url = 'https://raw.githubusercontent.com/MVR46/airbnb-data-madrid/main/listings.csv'
df = pd.read_csv(url)

# Display basic information
print(f"Dataset shape: {df.shape}")
pd.set_option('display.max_columns', None)
display(df.head())

Dataset shape: (26004, 79)


,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,availability_eoy,number_of_reviews_ly,estimated_occupancy_l365d,estimated_revenue_l365d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,21853,https://www.airbnb.com/rooms/21853,20250612050748,2025-06-26,city scrape,Bright and airy room,We have a quiet and sunny room with a good vie...,We live in a leafy neighbourhood with plenty o...,https://a0.muscache.com/pictures/68483181/87bc...,83531,https://www.airbnb.com/users/show/83531,Abdel,2010-02-21,"Madrid, Spain",EN-ES-FR\r\nEN\r\nHi everybody: I'm Abdel. I'm...,NaN,NaN,NaN,f,https://a0.muscache.com/im/users/83531/profile...,https://a0.muscache.com/im/users/83531/profile...,Aluche,2.0,2.0,"['email', 'phone']",t,t,"Madrid, Spain",Cármenes,Latina,40.40381,-3.74130,Private room in rental unit,Private room,1,1.0,1 bath,1.0,1.0,"[""Washer"", ""Hair dryer"", ""Coffee maker"", ""Oven...",$29.00,4,40,4.0,4.0,40.0,40.0,4.0,40.0,NaN,t,5,35,65,340,2025-06-26,33,0,0,164,0,0,0.0,2014-10-10,2018-07-15,4.58,4.72,4.56,4.75,4.82,4.21,4.67,NaN,f,2,0,2,0,0.25
1,30320,https://www.airbnb.com/rooms/30320,20250612050748,2025-06-27,previous scrape,Apartamentos Dana Sol,NaN,NaN,https://a0.muscache.com/pictures/336868/f67409...,130907,https://www.airbnb.com/users/show/130907,Dana,2010-05-24,"Madrid, Spain",Apartasol offers a network of several spacious...,NaN,NaN,NaN,f,https://a0.muscache.com/im/users/130907/profil...,https://a0.muscache.com/im/users/130907/profil...,Sol,3.0,6.0,"['email', 'phone']",t,f,NaN,Sol,Centro,40.41476,-3.70418,Entire rental unit,Entire home/apt,2,NaN,1 bath,1.0,NaN,"[""TV with standard cable"", ""Air conditioning"",...",NaN,5,50,5.0,7.0,50.0,50.0,5.1,50.0,NaN,t,2,32,62,337,2025-06-27,172,0,0,160,0,0,NaN,2010-07-06,2022-09-26,4.63,4.71,4.88,4.82,4.78,4.90,4.69,NaN,f,3,3,0,0,0.94
2,30959,https://www.airbnb.com/rooms/30959,20250612050748,2025-06-27,previous scrape,Beautiful loft in Madrid Center,Beautiful Loft 60m2 size just in the historica...,NaN,https://a0.muscache.com/pictures/78173471/835e...,132883,https://www.airbnb.com/users/show/132883,Angela,2010-05-26,"Madrid, Spain",Estoy empezando en Airbnb y deseo que mis hués...,NaN,NaN,NaN,f,https://a0.muscache.com/im/users/132883/profil...,https://a0.muscache.com/im/users/132883/profil...,Embajadores,1.0,4.0,"['email', 'phone']",t,f,NaN,Embajadores,Centro,40.41259,-3.70105,Entire loft,Entire home/apt,2,NaN,1 bath,1.0,NaN,"[""Washer"", ""Breakfast"", ""Essentials"", ""Kitchen...",NaN,3,730,3.0,3.0,730.0,730.0,3.0,730.0,NaN,NaN,0,0,0,0,2025-06-27,8,0,0,0,0,0,NaN,2015-05-12,2017-05-30,4.38,4.14,4.38,4.63,4.63,4.88,4.25,NaN,f,1,1,0,0,0.06
3,40916,https://www.airbnb.com/rooms/40916,20250612050748,2025-06-26,previous scrape,Apartasol Apartamentos Dana,NaN,NaN,https://a0.muscache.com/p

#EDA

In [ ]:
# Check for missing values
print("\nMissing values per column:")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', None)
print(df.isnull().sum())

# Display data types of each column
print("\nData types of each column:")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', None)
print(pd.DataFrame(df.dtypes, columns=['dtype']))


Missing values per column:
id                                                  0
listing_url                                         0
scrape_id                                           0
last_scraped                                        0
source                                              0
name                                                0
description                                       900
neighborhood_overview                           14846
picture_url                                         2
host_id                                             0
host_url                                            0
host_name                                          15
host_since                                         16
host_location                                    8069
host_about                                      12945
host_response_time                               4728
host_response_rate                               4728
host_acceptance_rate                             3985


#Remove useless features

In [ ]:
# --- Columns to keep for search/filtering ---
KEEP_COLS = [
    # Identifiers / URLs
    'id', 'listing_url', 'picture_url',
    # Location
    'neighbourhood_group_cleansed', 'neighbourhood_cleansed', 'latitude', 'longitude',
    # Type / capacity
    'room_type', 'property_type', 'accommodates', 'bathrooms', 'bathrooms_text', 'bedrooms', 'beds',
    # Amenities & policies
    'amenities', 'instant_bookable', 'has_availability',
    # Price & stay rules
    'price', 'minimum_nights', 'maximum_nights',
    # Aggregate availability
    'availability_30', 'availability_60', 'availability_90', 'availability_365',
    # Reputation
    'review_scores_rating', 'number_of_reviews',
    # Host signals (optional for ranking)
    'host_is_superhost', 'host_response_rate', 'host_acceptance_rate',
    # Text (for later NLP)
    'name', 'description', 'neighborhood_overview',
    # Useful dates
    'first_review', 'last_review'
]

# Keep only columns that actually exist in the CSV (avoid KeyError)
keep_existing = [c for c in KEEP_COLS if c in df.columns]

# Drop everything else
to_drop = sorted(col for col in df.columns if col not in keep_existing)
df_reduced = df.drop(columns=to_drop).copy()

print(f"Kept columns: {len(keep_existing)}")
print(f"Dropped columns: {len(to_drop)}")
print("Original shape:", df.shape, "-> Reduced shape:", df_reduced.shape)

# Quick peek
pd.set_option('display.max_columns', None)
display(df_reduced.head())


Kept columns: 34
Dropped columns: 45
Original shape: (26004, 79) -> Reduced shape: (26004, 34)


,id,listing_url,name,description,neighborhood_overview,picture_url,host_response_rate,host_acceptance_rate,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,first_review,last_review,review_scores_rating,instant_bookable
0,21853,https://www.airbnb.com/rooms/21853,Bright and airy room,We have a quiet and sunny room with a good vie...,We live in a leafy neighbourhood with plenty o...,https://a0.muscache.com/pictures/68483181/87bc...,NaN,NaN,f,Cármenes,Latina,40.40381,-3.74130,Private room in rental unit,Private room,1,1.0,1 bath,1.0,1.0,"[""Washer"", ""Hair dryer"", ""Coffee maker"", ""Oven...",$29.00,4,40,t,5,35,65,340,33,2014-10-10,2018-07-15,4.58,f
1,30320,https://www.airbnb.com/rooms/30320,Apartamentos Dana Sol,NaN,NaN,https://a0.muscache.com/pictures/336868/f67409...,NaN,NaN,f,Sol,Centro,40.41476,-3.70418,Entire rental unit,Entire home/apt,2,NaN,1 bath,1.0,NaN,"[""TV with standard cable"", ""Air conditioning"",...",NaN,5,50,t,2,32,62,337,172,2010-07-06,2022-09-26,4.63,f
2,30959,https://www.airbnb.com/rooms/30959,Beautiful loft in Madrid Center,Beautiful Loft 60m2 size just in the historica...,NaN,https://a0.muscache.com/pictures/78173471/835e...,NaN,NaN,f,Embajadores,Centro,40.41259,-3.70105,Entire loft,Entire home/apt,2,NaN,1 bath,1.0,NaN,"[""Washer"", ""Breakfast"", ""Essentials"", ""Kitchen...",NaN,3,730,NaN,0,0,0,0,8,2015-05-12,2017-05-30,4.38,f
3,40916,https://www.airbnb.com/rooms/40916,Apartasol Apartamentos Dana,NaN,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,NaN,NaN,f,Universidad,Centro,40.42247,-3.70577,Entire rental unit,Entire home/apt,2,NaN,1 bath,1.0,NaN,"[""TV"", ""Air conditioning"", ""Heating"", ""Elevato...",NaN,5,50,t,17,47,77,352,49,2010-11-01,2021-12-11,4.65,t
4,62423,https://www.airbnb.com/rooms/62423,MAGIC ARTISTIC HOUSE IN THE CENTER OF MADRID,INCREDIBLE HOME OF AN ARTIST SURROUNDED BY PAI...,DISTRICT WITH VERY GOOD VIBES IN THE MIDDLE OF...,https://a0.muscache.com/pictures/miso/Hosting-...,100%,100%,f,Justicia,Centro,40.41884,-3.69655,Private room in rental unit,Private room,4,1.5,1.5 shared baths,1.0,2.0,"[""Wine glasses"", ""Washer"", ""Dining table"", ""Ha...",$64.00,1,30,t,8,31,49,314,234,2018-05-10,2025-06-07,4.65,f


#Data transformation

In [ ]:
# Data transformation types
import numpy as np
import re

# Start from your reduced dataset
df = df_reduced.copy()

# ---------- Helpers ----------
def parse_currency(x):
    """Convert currency-like strings to float (e.g., '$1,234.00' -> 1234.00)."""
    if pd.isna(x):
        return np.nan
    s = re.sub(r'[^\d,.\-]', '', str(x))  # keep digits, comma, dot, minus
    # Remove thousands separators assuming US/Intl format -> 1,234.56
    s = s.replace(',', '')
    try:
        return float(s)
    except Exception:
        return np.nan

def parse_percent(s):
    return (pd.Series(s)
            .astype(str)
            .str.replace('%', '', regex=False)
            .replace(['nan', 'None', 'NaN'], np.nan)
            .astype(float))

def bathrooms_from_text(txt):
    """Extract numeric bathrooms from bathrooms_text. 'Half-bath' -> 0.5, '1.5 baths' -> 1.5."""
    if pd.isna(txt):
        return np.nan
    t = str(txt).lower()
    if 'half-bath' in t or 'half bath' in t:
        m = re.search(r'(\d+(\.\d+)?)', t)
        return float(m.group(1)) if m else 0.5
    m = re.search(r'(\d+(\.\d+)?)', t)
    return float(m.group(1)) if m else np.nan

def tf_to_bool(values, default=False):
    """
    Convert typical truthy/falsey tokens to python bool without using .fillna (avoids FutureWarning).
    Uses .where(...) to set defaults instead of .fillna(...).
    """
    mapping = {
        True: True, False: False,
        't': True, 'true': True, '1': True, 'y': True, 'yes': True,
        'f': False, 'false': False, '0': False, 'n': False, 'no': False
    }
    ser = pd.Series(values, dtype='object')
    # Normalize strings to lower; keep actual bools as-is
    norm = np.where(ser.isin([True, False]), ser, ser.astype(str).str.strip().str.lower())
    mapped = pd.Series(norm, dtype='object').map(mapping)
    # Use .where instead of .fillna to avoid silent downcasting warning
    if default is not None:
        mapped = mapped.where(pd.notna(mapped), default)
    return mapped.astype(bool)

# ---------- Parse key fields ----------
if 'price' in df.columns:
    df['price'] = df['price'].apply(parse_currency)

# fechas
for c in ['first_review', 'last_review']:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors='coerce')

# porcentajes
for c in ['host_response_rate', 'host_acceptance_rate']:
    if c in df.columns:
        df[c] = parse_percent(df[c])

# Convert object-like booleans to proper bools (if present)
for c in ['instant_bookable', 'host_is_superhost', 'has_availability']:
    if c in df.columns:
        df[c] = tf_to_bool(df[c], default=False)

# Derive bathrooms from bathrooms_text where missing
if {'bathrooms', 'bathrooms_text'}.issubset(df.columns):
    mask_missing_bath = df['bathrooms'].isna()
    df.loc[mask_missing_bath, 'bathrooms'] = df.loc[mask_missing_bath, 'bathrooms_text'].apply(bathrooms_from_text)

print("Transformations complete. Current shape:", df.shape)

Transformations complete. Current shape: (26004, 34)


#Remove rows where there are missing values on important features, and adapting other rows where missing values can be derived


In [ ]:
# Missing value management (elimination or handling)

# ---------- Config: strictness toggles ----------
DROP_IF_MISSING_PICTURE = True        # Set to False if a picture is not mandatory
STRICT_DROP_HAS_AVAILABILITY = False  # If True, drop rows where has_availability is missing; else set to False

# ---------- Define critical must-have features ----------
CRITICAL_MUST_HAVE = [
    'price',                       # parsed float
    'room_type', 'property_type',
    'accommodates',
    'neighbourhood_cleansed',
    'latitude', 'longitude', 'bathrooms','bathrooms_text', 'bedrooms', 'beds'
]
if STRICT_DROP_HAS_AVAILABILITY and 'has_availability' in df.columns:
    CRITICAL_MUST_HAVE.append('has_availability')

critical_existing = [c for c in CRITICAL_MUST_HAVE if c in df.columns]

# ---------- Drop rows with missing critical features ----------
before_rows = df.shape[0]
df = df.dropna(subset=critical_existing)

# Optional: ensure every listing has a picture for UX
if DROP_IF_MISSING_PICTURE and 'picture_url' in df.columns:
    df = df.dropna(subset=['picture_url'])

after_rows_critical = df.shape[0]
print(f"Rows dropped due to critical missing values: {before_rows - after_rows_critical}")
print("Shape after critical drops:", df.shape)


# ---------- Sanity checks (optional) ----------
assert df['price'].notna().all(), "There are still missing prices!"
assert df['accommodates'].notna().all(), "There are missing accommodates!"
assert df['latitude'].notna().all() and df['longitude'].notna().all(), "There are missing coordinates!"
assert df['bathrooms_text'].notna().all(), "There are listings missing bathrooms_text!"

Rows dropped due to critical missing values: 5993
Shape after critical drops: (20011, 34)


#Data quality and consistency + helpfull feature creations

In [ ]:
from collections import Counter

# Add-on cell: A + B (except amenity flags) + host rate bounding

# --- 0) Start from your current `df` produced above ---
assert 'df' in globals(), "Run the previous cells first to create `df`."

# --- 1) Deduplicate by listing id (A.1) ---
before = len(df)
if 'id' in df.columns:
    df = df.drop_duplicates(subset=['id']).copy()
print(f"[Dedup] Dropped {before - len(df)} duplicate rows by 'id'.")

# --- 2) Basic validity filters (A.2) ---
valid_masks = []

if 'price' in df.columns:
    valid_masks.append(df['price'] > 0)

if 'accommodates' in df.columns:
    valid_masks.append(df['accommodates'] >= 1)

if 'bathrooms' in df.columns:
    # allow 0.5 (half-bath). Treat NaN as invalid (should have been cleaned earlier).
    valid_masks.append(df['bathrooms'] >= 0.5)

for col in ['bedrooms', 'beds']:
    if col in df.columns:
        valid_masks.append(df[col] >= 0)

valid_mask = np.logical_and.reduce(valid_masks) if valid_masks else np.ones(len(df), dtype=bool)
before = len(df)
df = df.loc[valid_mask].copy()
print(f"[Validity] Dropped {before - len(df)} rows failing basic value checks.")

# --- 4) Min/Max nights sanity (A.4) ---
if {'minimum_nights', 'maximum_nights'}.issubset(df.columns):
    # Swap where min > max
    swap_mask = df['minimum_nights'] > df['maximum_nights']
    if swap_mask.any():
        mn = df.loc[swap_mask, 'maximum_nights']
        mx = df.loc[swap_mask, 'minimum_nights']
        df.loc[swap_mask, 'minimum_nights'] = mn.values
        df.loc[swap_mask, 'maximum_nights'] = mx.values
        print(f"[Nights] Swapped min/max nights for {swap_mask.sum()} rows.")
    # Clip to reasonable bounds
    df['minimum_nights'] = df['minimum_nights'].clip(lower=1, upper=365)
    df['maximum_nights'] = df['maximum_nights'].clip(lower=1, upper=999)

# --- 6) Categorical hygiene (B.3) ---
def clean_categorical(series):
    if series.dtype == 'O':
        return (series.astype(str)
                      .str.strip()
                      .str.replace(r'\s+', ' ', regex=True))
    return series

for c in ['room_type', 'property_type', 'neighbourhood_group_cleansed', 'neighbourhood_cleansed']:
    if c in df.columns:
        df[c] = clean_categorical(df[c])

# --- 7) Text hygiene (B.4) ---
for c in ['name', 'description', 'neighborhood_overview']:
    if c in df.columns:
        df[c] = (df[c].astype(str)
                       .str.replace(r'\s+', ' ', regex=True)
                       .str.strip())

# --- 8) Amenities normalization (B.1, but WITHOUT flags) ---
def parse_amenities_to_list(a):
    if pd.isna(a):
        return []
    s = str(a).strip()
    # Strip brackets/braces/quotes and split by comma
    s = s.replace('[','').replace(']','').replace('{','').replace('}','')
    parts = [p.strip().strip('"').strip("'").lower() for p in s.split(',') if p.strip()!='']
    # Deduplicate while preserving order
    seen, out = set(), []
    for p in parts:
        if p not in seen:
            seen.add(p)
            out.append(p)
    return out

if 'amenities' in df.columns:
    df['amenities_norm'] = df['amenities'].apply(parse_amenities_to_list)

# Global frequency table of exact amenity tokens
counter = Counter()
_ = df['amenities_norm'].apply(counter.update)
amenity_freq = (
    pd.DataFrame(counter.items(), columns=['amenity','count'])
      .sort_values('count', ascending=False)
      .reset_index(drop=True)
)

# Preview and keep artifacts for later use
display(amenity_freq.head(50))
canonical_amenities = amenity_freq['amenity'].tolist()
print(f"Unique amenity tokens found: {len(canonical_amenities)}")

# --- 9) Price-derived helpers (B.2) ---
if {'price', 'accommodates'}.issubset(df.columns):
    df['price_per_guest'] = (df['price'] / df['accommodates']).replace([np.inf, -np.inf], np.nan)

# --- 10) Host rates bounded to [0, 100] (C: selected item) ---
for c in ['host_response_rate', 'host_acceptance_rate']:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce').clip(lower=0, upper=100)

# --- 11) Type-casting after cleaning (A.5) ---
if 'accommodates' in df.columns:
    df['accommodates'] = pd.to_numeric(df['accommodates'], errors='coerce').astype('Int64')
for c in ['bedrooms', 'beds', 'number_of_reviews']:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')
        # bedrooms/beds should be integer counts; keep nullable Int64
        if c in ['bedrooms', 'beds']:
            df[c] = df[c].round(0).astype('Int64')
        else:
            df[c] = df[c].round(0).clip(lower=0).astype('Int64')
if 'bathrooms' in df.columns:
    df['bathrooms'] = pd.to_numeric(df['bathrooms'], errors='coerce').astype(float)

# --- 12) Finalize ---
df_ready = df.reset_index(drop=True).copy()
print("df_ready shape:", df_ready.shape)
print("df_ready columns:", sorted(df_ready.columns.tolist()))


[Dedup] Dropped 0 duplicate rows by 'id'.
[Validity] Dropped 137 rows failing basic value checks.


,amenity,count
0,wifi,18393
1,kitchen,17942
2,hot water,15976
3,bed linens,14297
4,dishes and silverware,14276
5,hair dryer,14256
6,iron,13903
7,hangers,13839
8,microwave,13590
9,refrigerator,13310


Unique amenity tokens found: 2701
df_ready shape: (19874, 36)
df_ready columns: ['accommodates', 'amenities', 'amenities_norm', 'availability_30', 'availability_365', 'availability_60', 'availability_90', 'bathrooms', 'bathrooms_text', 'bedrooms', 'beds', 'description', 'first_review', 'has_availability', 'host_acceptance_rate', 'host_is_superhost', 'host_response_rate', 'id', 'instant_bookable', 'last_review', 'latitude', 'listing_url', 'longitude', 'maximum_nights', 'minimum_nights', 'name', 'neighborhood_overview', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'number_of_reviews', 'picture_url', 'price', 'price_per_guest', 'property_type', 'review_scores_rating', 'room_type']


#NLP

#Cell 0 — Install & load core NLP models

In [ ]:
# === Cell 0 (SLIM): core deps + load models ===
%pip -q install -U spacy dateparser sentence-transformers torch scikit-learn

# Quita libs que suelen fijar versiones incompatibles (opcional pero recomendable)
%pip -q uninstall -y fastai torchvision torchaudio opencv-python opencv-contrib-python opencv-python-headless tsfresh || true

import os, torch
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["OMP_NUM_THREADS"] = "1"

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# spaCy EN
import spacy
try:
    nlp = spacy.load("en_core_web_sm")
except Exception:
    from spacy.cli import download
    download("en_core_web_sm")
    nlp = spacy.load("en_core_web_sm")
print("spaCy:", "en_core_web_sm")

# Sentence embeddings (English, small & fast)
from sentence_transformers import SentenceTransformer
embed_model = SentenceTransformer("all-MiniLM-L6-v2", device=device)
print("SBERT dims:", embed_model.get_sentence_embedding_dimension())


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 919.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.3/322.3 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SBERT dims: 384


#Cell 1 — Build label pools from your data & precompute embeddings

In [ ]:
# === Cell 1 (SLIM): pools + embeddings + amenity clustering & cleanup ===
import numpy as np
import pandas as pd
import re, unicodedata
from collections import Counter

assert 'df_ready' in globals(), "Run your cleaning pipeline first to create `df_ready`."
assert 'amenities_norm' in df_ready.columns, "`amenities_norm` is required."

df_search = df_ready.reset_index(drop=True).copy()

def strip_accents(s: str) -> str:
    if not isinstance(s, str): return ''
    return ''.join(ch for ch in unicodedata.normalize('NFD', s) if unicodedata.category(ch) != 'Mn')

def norm_txt(s: str) -> str:
    s = strip_accents(str(s)).lower().strip()
    return re.sub(r'\s+', ' ', s)

# --- Label pools ---
neigh_vals = sorted(set(df_search['neighbourhood_cleansed'].dropna().astype(str)))
group_vals = sorted(set(df_search['neighbourhood_group_cleansed'].dropna().astype(str)))
room_vals  = sorted(set(df_search['room_type'].dropna().astype(str)))
prop_vals  = sorted(set(df_search['property_type'].dropna().astype(str)))

neigh_norm = [norm_txt(x) for x in neigh_vals]
group_norm = [norm_txt(x) for x in group_vals]
room_norm  = [norm_txt(x) for x in room_vals]
prop_norm  = [norm_txt(x) for x in prop_vals]

print(f"Pools — neighborhoods:{len(neigh_vals)}, groups:{len(group_vals)}, room:{len(room_vals)}, prop:{len(prop_vals)}")

# --- Embedding util ---
def enc_norm_list(xs: list) -> np.ndarray:
    if not xs:
        return np.zeros((0, embed_model.get_sentence_embedding_dimension()), dtype=np.float32)
    v = embed_model.encode(xs, show_progress_bar=False, normalize_embeddings=True)
    return np.asarray(v, dtype=np.float32)

# --- Label embeddings ---
E_NEIGH = enc_norm_list(neigh_norm)
E_GROUP = enc_norm_list(group_norm)
E_ROOM  = enc_norm_list(room_norm)
E_PROP  = enc_norm_list(prop_norm)
print("Embeddings:", E_NEIGH.shape, E_GROUP.shape, E_ROOM.shape, E_PROP.shape)

# --- Amenity tokens (ALL) + embeddings ---
ALL_AMEN_TOKENS = sorted({norm_txt(x) for row in df_search['amenities_norm'] for x in (row or []) if x})
E_ALL_AMEN = enc_norm_list(ALL_AMEN_TOKENS)
print("Unique amenity tokens:", len(ALL_AMEN_TOKENS))

# --- Cluster amenities into canonical families ---
from sklearn.cluster import KMeans

M = E_ALL_AMEN.shape[0]
assert M == len(ALL_AMEN_TOKENS) and M > 0, "Amenity embeddings missing."

K = min(300, max(60, M // 8))
kmeans = KMeans(n_clusters=K, n_init=10, random_state=42)
amen_labels = kmeans.fit_predict(E_ALL_AMEN)
centroids = kmeans.cluster_centers_
centroids = (centroids / np.linalg.norm(centroids, axis=1, keepdims=True)).astype(np.float32)

AMEN_CLUSTER_ID = {tok: int(cid) for tok, cid in zip(ALL_AMEN_TOKENS, amen_labels)}

# Freq de tokens para escoger el canon de cada cluster
tok_freq = Counter()
_ = df_search['amenities_norm'].apply(lambda xs: tok_freq.update([norm_txt(x) for x in (xs or [])]))

cluster_members = {}
for tok, cid in AMEN_CLUSTER_ID.items():
    cluster_members.setdefault(cid, []).append(tok)

CLUSTER_CANON = {}
AMEN_CANON = {}
for cid, toks in cluster_members.items():
    toks_sorted = sorted(toks, key=lambda t: (-tok_freq.get(t, 0), len(t)))
    canon = toks_sorted[0]
    CLUSTER_CANON[cid] = canon
    for t in toks:
        AMEN_CANON[t] = canon

CANON_LIST = [CLUSTER_CANON[cid] for cid in sorted(CLUSTER_CANON.keys())]
E_CANON = centroids[np.array(sorted(CLUSTER_CANON.keys()))].astype(np.float32)

# Canon members set
CANON_MEMBERS = {}
for tok, canon in AMEN_CANON.items():
    CANON_MEMBERS.setdefault(canon, set()).add(tok)

# Columna canónica por listing (para filtrar más tarde)
df_search['amenities_canon'] = df_search['amenities_norm'].apply(
    lambda xs: sorted({AMEN_CANON.get(norm_txt(x)) for x in (xs or []) if AMEN_CANON.get(norm_txt(x))})
)

# --- Lexical helpers (reused everywhere) ---
def content_tokens(s: str):
    toks = re.findall(r"[a-zA-Z0-9]+", norm_txt(s))
    STOP = nlp.Defaults.stop_words
    GENERIC = {"city","center","centre","view","area","place","space","access","daily","day","night","room"}
    return {t for t in toks if len(t) >= 3 and t not in STOP and t not in GENERIC}

# Canon tokens (para gates léxicos)
CANON_TOKENS = {c: content_tokens(c) for c in CANON_LIST}

# --- Canonical amenities merge & cleanup ---
CANON_FREQ = {canon: sum(tok_freq.get(t, 0) for t in CANON_MEMBERS.get(canon, []))
              for canon in CANON_LIST}

GENERIC_MODIFIERS = {"standard", "estandar", "basic", "regular", "general"}

def label_merge_key(label: str):
    toks = content_tokens(label)
    toks = {t for t in toks if t not in GENERIC_MODIFIERS}
    return tuple(sorted(toks)) if toks else (norm_txt(label),)

def is_weird_unicode(label: str) -> bool:
    return bool(re.search(r'\\u[0-9a-fA-F]{4}', label))

def _prefer_label(lbls):
    def sort_key(lbl):
        ascii_ok = all(ord(ch) < 128 for ch in lbl)
        return (
            0 if not is_weird_unicode(lbl) else 1,
            0 if ascii_ok else 1,
            -CANON_FREQ.get(lbl, 0),
            len(lbl),
            norm_txt(lbl),
        )
    return sorted(lbls, key=sort_key)[0]

_groups = {}
for c in CANON_LIST:
    _groups.setdefault(label_merge_key(c), []).append(c)

CANON_MERGE = {}
for _, lbls in _groups.items():
    rep = _prefer_label(lbls)
    for l in lbls:
        CANON_MERGE[l] = rep

print(f"Amenity families: {len(CANON_LIST)} canonical labels from {M} tokens.")
print(f"Built CANON_MERGE for {len(CANON_MERGE)} canonical labels.")

# --- Geography helpers (gate barrios by group) ---
df_geo = (df_search[['neighbourhood_group_cleansed','neighbourhood_cleansed']]
          .dropna().drop_duplicates())
df_geo['g_norm'] = df_geo['neighbourhood_group_cleansed'].map(norm_txt)
df_geo['n_norm'] = df_geo['neighbourhood_cleansed'].map(norm_txt)

group_to_neigh = (df_geo.groupby('g_norm')['n_norm'].apply(set).to_dict())
neigh_to_group = {row.n_norm: row.g_norm for _, row in df_geo.iterrows()}

print(f"Groups: {len(group_to_neigh)} | Neighbourhoods: {len(neigh_to_group)}")

Pools — neighborhoods:128, groups:21, room:4, prop:60
Embeddings: (128, 384) (21, 384) (4, 384) (60, 384)
Unique amenity tokens: 2581
Amenity families: 300 canonical labels from 2581 tokens.
Built CANON_MERGE for 300 canonical labels.
Groups: 21 | Neighbourhoods: 128


#Cell 2 — Parser (smaller TOP-K on CPU, higher thresholds)

In [ ]:
# === Cell 2 (SLIM): parser (amenities/neighborhoods/types) ===
from typing import List, Tuple, Optional, Dict, Any
import numpy as np
import pandas as pd
import re, dateparser, spacy

def extract_price_range_en(text: str) -> Tuple[Optional[float], Optional[float]]:
    t = norm_txt(text); CUR = r"(?:€|\$|eur|usd)"
    m = re.search(rf"(?:between|from)\s*{CUR}?\s*(\d{{2,6}})\s*(?:and|to|-)\s*{CUR}?\s*(\d{{2,6}})", t)
    if m:
        lo, hi = float(m.group(1)), float(m.group(2))
        return (min(lo, hi), max(lo, hi))
    m = re.search(rf"(?:up to|under|below|max(?:imum)?)\s*{CUR}?\s*(\d{{2,6}})", t)
    if m: return (None, float(m.group(1)))
    m = re.search(rf"{CUR}?\s*(\d{{2,6}})\s*(?:per\s*night|/night)", t)
    if m: return (None, float(m.group(1)))
    m = re.search(rf"{CUR}\s*(\d{{2,6}})", t)
    if m: return (None, float(m.group(1)))
    return (None, None)

def extract_guests_nights_en(text: str) -> Tuple[Optional[int], Optional[int]]:
    t = norm_txt(text)
    g = re.search(r"(\d{1,3})\s*(?:guests?|people|adults|persons?)", t)
    n = re.search(r"(\d{1,3})\s*(?:nights?|night)", t)
    return (int(g.group(1)) if g else None, int(n.group(1)) if n else None)

def maybe_parse_dates_en(text: str) -> Tuple[Optional[pd.Timestamp], Optional[pd.Timestamp], Optional[int]]:
    m = re.search(r"from\s+([^\n,;]+?)\s+to\s+([^\n,;]+)", text, flags=re.IGNORECASE)
    if m:
        d1 = dateparser.parse(m.group(1), languages=['en'])
        d2 = dateparser.parse(m.group(2), languages=['en'])
        if d1 and d2:
            d1 = pd.to_datetime(d1).normalize()
            d2 = pd.to_datetime(d2).normalize()
            if d2 <= d1: d2 = d1 + pd.Timedelta(days=1)
            return d1, d2, int((d2 - d1).days)
    return None, None, None

def noun_phrases_and_ngrams(text: str, nlp_model) -> List[str]:
    doc = nlp_model(text)
    chunks = [nc.text for nc in doc.noun_chunks]
    toks = [t.text for t in doc if t.pos_ in {"NOUN","PROPN"} and not t.is_stop]
    bigrams = [' '.join([toks[i], toks[i+1]]) for i in range(len(toks)-1)]
    cands = list(dict.fromkeys(chunks + toks + bigrams))
    return [c for c in cands if c.strip()]

def best_two(a: np.ndarray) -> Tuple[int, float, float]:
    if a.size == 0: return -1, -1.0, -1.0
    j = int(np.argmax(a))
    m1 = float(a[j])
    if a.size == 1:
        return j, m1, -1.0
    b = a.copy(); b[j] = -np.inf
    m2 = float(b.max())
    return j, m1, m2

def canonical_amenities_from_query(
    query: str,
    th_abs: float = 0.60,
    th_margin: float = 0.06
) -> List[str]:
    if not CANON_LIST:
        return []
    phrases = noun_phrases_and_ngrams(query, nlp)
    if not phrases:
        return []

    phrases_norm = [norm_txt(p) for p in phrases]
    Q = embed_model.encode(phrases_norm, normalize_embeddings=True)
    sims = Q @ E_CANON.T  # [n_chunks, n_canon]

    q_tokens = content_tokens(query)
    hits = set()

    for i, p in enumerate(phrases):
        j, m1, m2 = best_two(sims[i])
        if j < 0:
            continue
        canon = CANON_LIST[j]
        canon_tokens = CANON_TOKENS.get(canon, set())
        phrase_tokens = content_tokens(p)

        # A) aceptar si pasa umbral+margen y gate léxico por frase
        if m1 >= th_abs and (m2 < 0 or m1 - m2 >= th_margin):
            if phrase_tokens & canon_tokens:
                hits.add(canon)
                continue
        # B) fallback: gate léxico a nivel de query (ej. "parking")
        if q_tokens & canon_tokens:
            hits.add(canon)

    if not hits:
        return []

    # Merge de near-duplicates (e.g., "refrigerator" vs "est\u00C1ndar refrigerator")
    merged = {CANON_MERGE.get(h, h) for h in hits}

    # Preferir la variante “limpia”
    key_to_lbls = {}
    for h in merged:
        key_to_lbls.setdefault(label_merge_key(h), []).append(h)

    cleaned = set()
    for key, lbls in key_to_lbls.items():
        if len(lbls) == 1:
            cleaned.add(lbls[0])
        else:
            cleaned.add(_prefer_label(lbls))

    ordered = sorted(
        cleaned,
        key=lambda label: (0 if all(ord(ch) < 128 for ch in label) else 1, len(label), norm_txt(label))
    )
    return ordered

def neighborhoods_groups_from_query(query: str,
                                    thr_neigh: float = 0.68,
                                    thr_group: float = 0.60) -> Tuple[List[str], List[str]]:
    phrases = noun_phrases_and_ngrams(query, nlp) + [query]
    phrases_norm = [norm_txt(p) for p in phrases]
    Q = embed_model.encode(phrases_norm, normalize_embeddings=True)

    neighs, groups = set(), set()
    for v, raw in zip(Q, phrases):
        s_g = v @ E_GROUP.T
        jg, g1, g2 = best_two(s_g)
        if g1 >= thr_group:
            groups.add(group_norm[jg])

        s_n = v @ E_NEIGH.T
        jn, n1, n2 = best_two(s_n)
        if n1 >= thr_neigh and (n2 < 0 or n1 - n2 >= 0.06):
            if content_tokens(raw) & set(re.findall(r"[a-z0-9]+", neigh_norm[jn])):
                neighs.add(neigh_norm[jn])

    qn = norm_txt(query)
    for g in group_norm:
        if g in qn: groups.add(g)
    for n in neigh_norm:
        if n in qn: neighs.add(n)

    if groups:
        allowed = set().union(*[group_to_neigh.get(g, set()) for g in groups])
        neighs = {n for n in neighs if n in allowed}

    return list(neighs), list(groups)

# Per-label token bags (para gate léxico de tipos)
ROOM_CONTENT_TOKENS = {lbl: content_tokens(lbl) for lbl in room_norm}
PROP_CONTENT_TOKENS = {lbl: content_tokens(lbl) for lbl in prop_norm}

def type_from_query_by_centroid(
    query: str,
    labels_norm: List[str],
    E: np.ndarray,
    TOKENS_MAP: Dict[str, set],
    th_abs: float = 0.60,
    th_margin: float = 0.05
) -> Optional[str]:
    phrases = noun_phrases_and_ngrams(query, nlp) + [query]
    if not phrases:
        return None

    P_norm = [norm_txt(p) for p in phrases]
    Q = embed_model.encode(P_norm, normalize_embeddings=True)
    qtok = content_tokens(query)

    best_label, best_score = None, -1.0
    for i, raw in enumerate(phrases):
        sims = Q[i] @ E.T
        j, m1, m2 = best_two(sims)
        if j < 0:
            continue

        lbl = labels_norm[j]
        gate_tokens = TOKENS_MAP.get(lbl, set())
        if (m1 >= th_abs) and (m2 < 0 or m1 - m2 >= th_margin) and (content_tokens(raw) & gate_tokens or qtok & gate_tokens):
            if m1 > best_score:
                best_label, best_score = lbl, m1

    return best_label

def parse_user_query_ml(query: str) -> Dict[str, Any]:
    amenities_canon = canonical_amenities_from_query(query)
    neighbourhoods, neigh_groups = neighborhoods_groups_from_query(query)

    room_type = type_from_query_by_centroid(query, room_norm, E_ROOM, ROOM_CONTENT_TOKENS, th_abs=0.58, th_margin=0.05)
    property_type = type_from_query_by_centroid(query, prop_norm, E_PROP, PROP_CONTENT_TOKENS, th_abs=0.58, th_margin=0.05)

    guests, nights = extract_guests_nights_en(query)
    price_min, price_max = extract_price_range_en(query)
    checkin, checkout, nights_dates = maybe_parse_dates_en(query)
    if nights is None and nights_dates is not None:
        nights = nights_dates

    return {
        "raw_query": query,
        "amenities_all": amenities_canon,
        "neighbourhoods": neighbourhoods,
        "neigh_groups": neigh_groups,
        "room_type": room_type,
        "property_type": property_type,
        "guests": guests,
        "nights": nights,
        "price_min": price_min,
        "price_max": price_max,
        "checkin": checkin,
        "checkout": checkout,
        "nlp": "EN-only: amenity centroid+lexical, group-gated barrios, types by centroid+lexical"
    }

#Cell 3 — Semantic amenity satisfaction + filters

In [ ]:
# === Cell 3 (SLIM): filters ===
from typing import List, Dict, Any

def amenities_family_satisfied(list_canon: List[str], required_canon: List[str]) -> bool:
    if not required_canon:
        return True
    if not list_canon:
        return False
    aset = set(list_canon)
    return all(rc in aset for rc in required_canon)

def apply_filters_ml(df: pd.DataFrame, spec: Dict[str, Any]) -> pd.DataFrame:
    m = pd.Series(True, index=df.index)

    if spec.get('neigh_groups'):
        m &= df['neighbourhood_group_cleansed'].astype(str).map(norm_txt).isin(spec['neigh_groups'])
    if spec.get('neighbourhoods'):
        m &= df['neighbourhood_cleansed'].astype(str).map(norm_txt).isin(spec['neighbourhoods'])

    if spec.get('guests') is not None:
        m &= df['accommodates'] >= spec['guests']
    if spec.get('nights') is not None and {'minimum_nights','maximum_nights'}.issubset(df.columns):
        m &= (df['minimum_nights'] <= spec['nights']) & (df['maximum_nights'] >= spec['nights'])

    if spec.get('price_min') is not None:
        m &= df['price'] >= spec['price_min']
    if spec.get('price_max') is not None:
        m &= df['price'] <= spec['price_max']

    if spec.get('room_type') is not None:
        m &= df['room_type'].astype(str).map(norm_txt) == norm_txt(spec['room_type'])
    if spec.get('property_type') is not None:
        m &= df['property_type'].astype(str).map(norm_txt).str.contains(norm_txt(spec['property_type']), regex=False)

    if spec.get('amenities_all'):
        if 'amenities_canon' not in df.columns:
            raise ValueError("Run amenity canonicalization first (Cell 1).")
        m &= df['amenities_canon'].apply(lambda xs: amenities_family_satisfied(xs or [], spec['amenities_all']))

    if 'has_availability' in df.columns:
        m &= df['has_availability'] == True

    return df.loc[m].copy()

#Cell 4 - Cache listing embeddings (compute once)

In [ ]:
# === Cell 4 (SLIM): cache listing embeddings ===
from tqdm import tqdm
import numpy as np
import os

def _concat_text(row: pd.Series) -> str:
    name = str(row.get('name', '') or '').strip()
    desc = str(row.get('description', '') or '').strip()
    if not desc and 'neighborhood_overview' in row:
        desc = str(row.get('neighborhood_overview', '') or '').strip()
    txt = f"{name}. {desc}".strip()
    return txt if txt else name

TEXTS = df_search.apply(_concat_text, axis=1).tolist()
D = embed_model.get_sentence_embedding_dimension()
N = len(TEXTS)

EMB_PATH = "/content/LISTING_EMB_EN.npy"

LISTING_EMB = None
if os.path.exists(EMB_PATH):
    try:
        cached = np.load(EMB_PATH)
        if cached.shape == (N, D):
            LISTING_EMB = cached
            print(f"Loaded cached embeddings: {LISTING_EMB.shape}")
        else:
            print(f"Cache shape {cached.shape} != ({N},{D}). Will recompute.")
    except Exception as e:
        print("Failed to load cache; will recompute:", e)

if LISTING_EMB is None:
    BATCH = 256 if device == "cuda" else 64
    LISTING_EMB = np.zeros((N, D), dtype=np.float32)
    for i in tqdm(range(0, N, BATCH), desc="Encoding listings"):
        batch = TEXTS[i:i+BATCH]
        LISTING_EMB[i:i+BATCH] = embed_model.encode(
            batch, batch_size=BATCH, show_progress_bar=False, normalize_embeddings=True
        ).astype("float32")
    np.save(EMB_PATH, LISTING_EMB)
    print(f"Saved embeddings to {EMB_PATH}")

print("LISTING_EMB ready:", LISTING_EMB.shape)

Encoding listings:  43%|████▎     | 135/311 [14:00<16:59,  5.80s/it]

#Cell 5 — Lexical backend (fast default = scikit-learn; optional gensim)

In [ ]:
# === Cell 5 (SLIM): TF-IDF lexical backend + reranker ===
import re, numpy as np, pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize

def simple_tokenize(s: str):
    s = norm_txt(s)
    return re.findall(r"[a-zA-Z0-9_]+", s)

tfidf_vec = TfidfVectorizer(
    tokenizer=simple_tokenize,
    preprocessor=None,
    token_pattern=None,
    lowercase=False,
    min_df=2,
    max_features=200000
)
X = tfidf_vec.fit_transform(TEXTS)
X = normalize(X, norm="l2", copy=False)
print("TF-IDF (sklearn):", X.shape)

def lexical_similarities(query: str) -> np.ndarray:
    qv = tfidf_vec.transform([query])
    qv = normalize(qv, norm="l2", copy=False)
    sims = (X @ qv.T).toarray().ravel()
    if sims.size and sims.max() > sims.min():
        sims = (sims - sims.min()) / (sims.max() - sims.min())
    return sims.astype(np.float32)

def rerank_semantic_with_lex(candidates: pd.DataFrame, query: str,
                             w_sem=0.55, w_lex=0.20, w_rating=0.15, w_price=0.10) -> pd.DataFrame:
    if candidates.empty:
        return candidates

    qv = embed_model.encode([norm_txt(query)], normalize_embeddings=True)[0].astype('float32')
    sem = (LISTING_EMB[candidates.index.values] @ qv)
    sem = np.clip((sem + 1) / 2, 0, 1)

    lex_all = lexical_similarities(query)
    lex = lex_all[candidates.index.values]

    rating = candidates.get('review_scores_rating', pd.Series(0.0, index=candidates.index)).fillna(0.0) / 100.0
    ppg = candidates.get('price_per_guest', pd.Series(np.nan, index=candidates.index))
    ppg = ppg.fillna(candidates['price'] / candidates['accommodates'].replace(0, np.nan))
    pmin, pmax = ppg.min(), ppg.max()
    price = ((ppg - pmin) / (pmax - pmin + 1e-9)).clip(0, 1)

    score = w_sem*sem + w_lex*lex + w_rating*rating + w_price*(1.0 - price)

    out = candidates.copy()
    out['__score'] = score
    out = out.sort_values(['__score','review_scores_rating','price'],
                          ascending=[False, False, True], kind='mergesort')
    return out.drop(columns=['__score'])

TF-IDF (sklearn): (19874, 8589)


#Cell 6 — Public search + quick smoke test (fast)

In [ ]:
from typing import Tuple

def natural_language_search(df: pd.DataFrame, query: str, top_n: int = 50) -> Tuple[pd.DataFrame, Dict[str, Any]]:
    spec = parse_user_query_ml(query)
    filtered = apply_filters_ml(df, spec)
    ranked = rerank_semantic_with_lex(filtered, query)
    plan = {
        "interpreted": {k: (str(v) if isinstance(v, pd.Timestamp) else v) for k, v in spec.items()},
        "counts": {"filtered": int(len(filtered)), "returned": int(min(top_n, len(ranked)))},
        "notes": "English-only: ZSC (DistilBERT MNLI) + SBERT prefilter + semantic amenity satisfaction + TF-IDF reranking. Exact-date availability needs calendar.csv."
    }
    return ranked.head(top_n), plan

q = "I want an loft in Salamanca or Madrid city center for 2 people, 3 nights, up to €100 per night, with wifi, and parking."
res, plan = natural_language_search(df_search, q, top_n=15)
print(plan["interpreted"])
display(res[['id','neighbourhood_group_cleansed','neighbourhood_cleansed','room_type','property_type','accommodates','price','amenities_norm']].head(10))


{'raw_query': 'I want an loft in Salamanca or Madrid city center for 2 people, 3 nights, up to €100 per night, with wifi, and parking.', 'amenities_all': ['wifi', 'free street parking'], 'neighbourhoods': [], 'neigh_groups': ['salamanca', 'centro'], 'room_type': None, 'property_type': 'entire loft', 'guests': 2, 'nights': 3, 'price_min': None, 'price_max': 100.0, 'checkin': None, 'checkout': None, 'nlp': 'EN-only: amenity centroid+lexical, group-gated barrios, types by centroid+lexical'}


,id,neighbourhood_group_cleansed,neighbourhood_cleansed,room_type,property_type,accommodates,price,amenities_norm
17196,1353456107855734758,Centro,Embajadores,Entire home/apt,Entire loft,4,99.0,"[wine glasses, washer, hair dryer, room-darken..."
210,1346747,Centro,Embajadores,Entire home/apt,Entire loft,4,88.0,"[washer, hair dryer, dishes and silverware, ho..."
1115,15514855,Centro,Universidad,Entire home/apt,Entire loft,4,55.0,"[washer, paid parking on premises, hair dryer,..."
2026,23931701,Centro,Palacio,Entire home/apt,Entire loft,2,52.0,"[washer, hair dryer, coffee maker, portable fa..."
1816,22443198,Centro,Embajadores,Entire home/apt,Entire loft,2,74.0,"[wine glasses, hair dryer, coffee maker, host ..."
10648,1009444371398077730,Salamanca,Guindalera,Entire home/apt,Entire loft,4,77.0,"[wine glasses, paid street parking off premise..."
401,4596254,Centro,Embajadores,Entire home/apt,Entire loft,4,89.0,"[hair dryer, coffee maker, room-darkening shad..."
5200,52814137,Centro,Justicia,Entire home/apt,Entire loft,2,76.0,"[washer, dining table, hair dryer, room-darken..."
1320,17152296,Centro,Embajadores,Entire home/apt,Entire loft,2,79.0,"[cooking basics, wine glasses, host greets you..."
479,6307542,Centro,Palacio,Entire home/apt,Entire loft,4,80.0,"[washer, dining table, paid parking on premise..."


In [ ]:
# === Quick test cell: try a new English query ===
# Edit the `q` string and re-run this cell.

q = "Looking for a apartment in Chamberi for 4 guests, 4 nights, between 100 and 250€ per night, with wifi, kitchen, iron, shampoo and washer."

# 1) Parse the query into a spec
spec = parse_user_query_ml(q)
print("Interpreted spec:")
for k in ["neigh_groups","neighbourhoods","room_type","property_type","amenities_all",
          "guests","nights","price_min","price_max","checkin","checkout"]:
    print(f"  {k}: {spec.get(k)}")

# 2) Apply filters
filtered = apply_filters_ml(df_search, spec)
print(f"\nFiltered candidates: {len(filtered)}")

# 3) Rerank and show top results
ranked = rerank_semantic_with_lex(filtered, q)
print(f"Returned (showing up to 10): {min(10, len(ranked))}\n")
display(ranked.head(10)[[
    "id","neighbourhood_group_cleansed","neighbourhood_cleansed",
    "room_type","property_type","accommodates","price","amenities_canon"
]])


Interpreted spec:
  neigh_groups: ['chamberi']
  neighbourhoods: []
  room_type: None
  property_type: entire serviced apartment
  amenities_all: ['iron', 'wifi', 'washer', 'kitchen', 'shampoo']
  guests: 4
  nights: 4
  price_min: 100.0
  price_max: 250.0
  checkin: None
  checkout: None

Filtered candidates: 2
Returned (showing up to 10): 2



,id,neighbourhood_group_cleansed,neighbourhood_cleansed,room_type,property_type,accommodates,price,amenities_canon
2729,31641111,Chamberí,Trafalgar,Entire home/apt,Entire serviced apartment,4,200.0,"[air conditioning, bed linens, clothing storag..."
2730,31641127,Chamberí,Trafalgar,Entire home/apt,Entire serviced apartment,4,242.0,"[air conditioning, baking sheet, bed linens, c..."
